In [ ]:
# here we find the overlap genes in human brain
import pandas as pd
import scanpy as sc
import sys

In [ ]:
meta = pd.read_csv('/workspaces/stimage/data/meta_all_gene.csv')
path = "/workspaces/stimage/data/raw"

In [ ]:
data = meta.loc[(meta['species'] == 'human') & (meta['tissue'] == 'lung'), :]
data.index = range(len(data.index))

for i in range(len(data.index)):
    gene_exp_slide = pd.read_csv(
        f'{path}/{data.tech[i]}/gene_exp/{data.slide[i]}_count.csv',
        sep=',',
        nrows=1,
        index_col=0)
    if i == 0:
        gene_name_overlap = gene_exp_slide.columns
    else:
        gene_name_overlap = gene_name_overlap.intersection(
            gene_exp_slide.columns)
    print(i)


In [ ]:
len(gene_name_overlap)

In [ ]:
import dask.dataframe as dd

# Initialize an empty Dask DataFrame
gene_exp = None

for index in range(len(data)):
    slide = data['slide'][index]
    tech = data['tech'][index]

    # Read the CSV using Dask
    gene_exp_slide = dd.read_csv(
        f'{path}/{tech}/gene_exp/{slide}_count.csv',
        sep=',',
        assume_missing=True,
        sample=10**7  # Increase this number if needed
    )

    # Filter the columns based on 'gene_name_overlap'
    gene_exp_slide_filtered = gene_exp_slide.loc[:, gene_name_overlap]

    if gene_exp is None:
        gene_exp = gene_exp_slide_filtered
    else:
        gene_exp = dd.concat([gene_exp, gene_exp_slide_filtered])

# Compute the Dask DataFrame (if needed)
gene_exp = gene_exp.compute()